In [29]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import pandas as pd
import numpy as np

import proyeccion_rdr.features as features
import proyeccion_rdr.produccion.a05_unidades_de_apoyo as ua
from proyeccion_rdr.produccion.a04_ambulatorio import (
    leer_consultas_medicas
)


pd.set_option("display.max_columns", None)

ANIOS_POBLACION = [str(i) for i in range(2017, 2036)]

## Leer insumos de Apoyo Diagnostico

In [31]:
# Lee la proyeccion de consultas medicas
RUTA_CONSULTAS = "../data/interim/3.0_estimacion_boxes_medicos_RDR.xlsx"
consultas_medicas_proyectadas = features.leer_proyeccion_consultas_medicas_especialidad(
    RUTA_CONSULTAS, ANIOS_POBLACION
)

# Lee la cantidad de DCO proyectados
RUTA_DCO = "../data/interim/1.0_estimacion_atencion_cerrada_RDR.xlsx"
dco_proyectados = features.leer_dco_proyectados(RUTA_DCO, ANIOS_POBLACION)

# Lee la cantidad de consultas de urgencias
RUTA_URGENCIAS = "../data/interim/4.0_estimacion_consultas_urgencia.xlsx"
consultas_c1_c2_c3, consultas_c4_c5, consultas_urgencia_proyectadas = (
    features.leer_consultas_urgencia_proyectadas(RUTA_URGENCIAS)
)

In [56]:
# Define las trazadoras a omitir
TRAZADORAS_A_OMITIR = []

# Lee los casos de todos los macroprocesos
RUTA_ARCHIVOS = "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx"
_, casos_macroproceso = features.leer_casos_macroprocesos(RUTA_ARCHIVOS)
casos_macroproceso = casos_macroproceso.query("Diagnostico not in @TRAZADORAS_A_OMITIR")

# Obtiene los casos de area de influencia y hosptializados
casos_area_de_influencia = casos_macroproceso.query("tipo_paciente == 'area_de_influencia'")
casos_hospitalizados_proyectados = casos_macroproceso.query("tipo_paciente == 'hospitalizados'")

# Aisla solamente los casos de cada macroproceso
casos_area_de_influencia = casos_area_de_influencia[ANIOS_POBLACION]
casos_hospitalizados_proyectados = casos_hospitalizados_proyectados[ANIOS_POBLACION]

# Obtiene los casos netamente de atencion abierta
casos_atencion_abierta = casos_area_de_influencia.sub(
    casos_hospitalizados_proyectados, fill_value=0
)
casos_atencion_abierta = casos_atencion_abierta.query("`2017` > 0")

# Consolida la cantidad de casos
casos_area_de_influencia = casos_area_de_influencia.sum()
casos_hospitalizados_proyectados = casos_hospitalizados_proyectados.sum()
casos_atencion_abierta = casos_atencion_abierta.sum()

In [32]:
# Define las rutas de los archivos
ruta_raw = "../data/raw/"

# Define las rutas de la base de CAE y GRD
ruta_cae = ruta_raw + "6_ambulatorio/df_procesada_consultas.csv"
ruta_grd = ruta_raw + "7_egresos_grd/df_procesada_grd.csv"
ruta_urgencias = ruta_raw + "8_urgencias/df_procesada_urgencias.csv"

# Define las rutas de los archivos de apoyo diagnostico
ruta_apoyo = ruta_raw + "9_unidades_de_apoyo/"

ruta_laboratorio = ruta_apoyo + "df_procesada_laboratorio_clinico.csv"
ruta_farmacia = ruta_farmacia_amb = ruta_apoyo + "df_procesada_farmacia.csv"
ruta_imagenologia = ruta_apoyo + "df_procesada_imagenologia.csv"
ruta_umt = ruta_apoyo + "df_procesada_umt.csv"

In [33]:
# Lee CAE y GRD
consultas_medicas = leer_consultas_medicas(ruta_cae)
df_grd = ua.leer_grd(ruta_grd)

C:\Users\jarojas\Desktop\Javier\SSMN\EPH - RDR\2.0-proyeccion-rdr\proyeccion_rdr\produccion\a04_ambulatorio.py:90: DtypeWarning: Columns (19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta, dtype={"id_paciente": str})
C:\Users\jarojas\Desktop\Javier\SSMN\EPH - RDR\2.0-proyeccion-rdr\proyeccion_rdr\produccion\a05_unidades_de_apoyo.py:12: DtypeWarning: Columns (47,48,49,50,51,52,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,220) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta, dtype={"id_paciente": str})


In [34]:
# Lee las unidades de apoyo
df_laboratorio, df_laboratorio_hosp, df_laboratorio_amb = ua.leer_laboratorio(ruta_laboratorio)
df_farmacia, df_farmacia_hosp, df_farmacia_amb = ua.leer_farmacia(ruta_farmacia)
df_imag, df_imag_hosp, df_imag_amb, df_imag_urg = ua.leer_imagenologia(ruta_imagenologia)

In [35]:
# Define la agrupacion a calcular
AGRUPACION_EXAMENES = ["ano", "id_paciente"]

# Obtiene los dco totales y consultas por pacientes por anio
consultas_por_pacientes = consultas_medicas.groupby(AGRUPACION_EXAMENES)["fecha_atencion"].count()
dco_por_pacientes = df_grd.groupby(AGRUPACION_EXAMENES)["estancia"].sum()

# Obtiene los examanes por paciente de laboratorio
examenes_ppct_lab_hosp = df_laboratorio_hosp.groupby(AGRUPACION_EXAMENES)["examen"].count()
examenes_ppct_lab_amb = df_laboratorio_amb.groupby(AGRUPACION_EXAMENES)["examen"].count()

# Obtiene las prescripciones por pacientes de farmacia
prescr_ppct_farmacia_hosp = df_farmacia_hosp.groupby(AGRUPACION_EXAMENES)["nombre_articulo"].count()
prescr_ppct_farmacia_amb = df_farmacia_amb.groupby(AGRUPACION_EXAMENES)["nombre_articulo"].count()

In [36]:
# Obtiene los examenes por DCO y consultas para laboratorio
_, examenes_por_dco_lab = ua.calcular_examenes_por_atencion(
    dco_por_pacientes, examenes_ppct_lab_hosp, "estancia", "examen"
)
_, examenes_por_consulta_lab = ua.calcular_examenes_por_atencion(
    consultas_por_pacientes, examenes_ppct_lab_amb, "fecha_atencion", "examen"
)

# Obtiene los examenes por DCO y consultas para laboratorio
_, prescr_por_dco = ua.calcular_examenes_por_atencion(
    dco_por_pacientes, prescr_ppct_farmacia_hosp, "estancia", "nombre_articulo"
)
_, prescr_por_consulta = ua.calcular_examenes_por_atencion(
    consultas_por_pacientes, prescr_ppct_farmacia_amb, "fecha_atencion", "nombre_articulo"
)

## Laboratorio Clinico

En este apartado se quiere saber cuantos examenes de Laboratorio Clinico se haran

In [37]:
print("Laboratorio Clinico - AC - Examenes por DCO")
display(examenes_por_dco_lab)

print("Laboratorio Clinico - AA - Examenes por CME")
display(examenes_por_consulta_lab)

Laboratorio Clinico - AC - Examenes por DCO


,count,id_paciente,mean,std,min,25%,50%,75%,max
ano,,,,,,,,,
2017,6781.0,14336,5.906895,13.123988,0.0,0.000000,2.900000,8.000000,677.000000
2018,7050.0,15511,5.405699,8.059222,0.0,0.000000,2.750000,7.770833,148.000000
2019,7255.0,15704,5.869289,8.475399,0.0,0.000000,3.250000,8.174242,189.800000
2020,4008.0,11966,7.675083,11.139853,0.0,0.636364,4.500000,10.719925,319.000000
2021,4544.0,18269,7.702273,9.850032,0.0,1.000000,4.466667,11.000000,162.000000
2022,5505.0,21862,7.507892,8.870251,0.0,1.285714,4.714286,10.500000,134.571429
2023,6183.0,15579,6.596584,8.709555,0.0,1.000000,4.000000,9.000000,173.777778
2024,6342.0,15844,6.794908,8.613264,0.0,0.878472,4.333333,9.592857,139.000000


Laboratorio Clinico - AA - Examenes por CME


,count,id_paciente,mean,std,min,25%,50%,75%,max
ano,,,,,,,,,
2017,33139.0,8068,0.595854,1.692162,0.0,0.0,0.0,0.000000,52.0
2018,34236.0,8583,0.614689,1.705245,0.0,0.0,0.0,0.000000,24.0
2019,35828.0,8872,0.637850,1.760020,0.0,0.0,0.0,0.000000,27.0
2020,23673.0,6245,0.823679,2.334450,0.0,0.0,0.0,0.000000,29.0
2021,25635.0,8999,0.959706,2.439627,0.0,0.0,0.0,0.500000,37.0
2022,30494.0,9599,0.902342,2.271074,0.0,0.0,0.0,0.500000,37.0
2023,33188.0,9605,0.913539,2.271995,0.0,0.0,0.0,0.500000,32.0
2024,35259.0,9272,0.830948,2.212097,0.0,0.0,0.0,0.142857,30.0


Como se ve, la cantidad de examenes por CME es muy baja, por lo que se ocupara el estandar MINSAL.

In [38]:
# Define constantes
CANTIDAD_EXAMENES_AC_A_HACER = examenes_por_dco_lab["75%"].max()
CANTIDAD_EXAMENES_AA_A_HACER = 3  # Estandar MINSAL
CANTIDAD_EXAMENES_URG_A_HACER = 2.5  # Estandar MINSAL
metricas_para_proyectar_laboratorio = pd.DataFrame(
    {
        "examenes_por_dco": [CANTIDAD_EXAMENES_AC_A_HACER],
        "examenes_por_consulta": [CANTIDAD_EXAMENES_AA_A_HACER],
    }
)

# Obtiene le cantidad de examenes de laboratorio requeridos para atencion cerrada
examenes_laboratorio_ac = dco_proyectados * CANTIDAD_EXAMENES_AC_A_HACER

# Obtiene la cantidad de examenes de laboratorio requeridos para atencion abierta
examanes_laboratorio_aa = consultas_medicas_proyectadas * CANTIDAD_EXAMENES_AA_A_HACER

# Obtiene la cantidad de examenes de laboratorio requeridos para atencion de urgencia
examenes_laboratorio_urgencia = consultas_urgencia_proyectadas * CANTIDAD_EXAMENES_URG_A_HACER

# Suma la cantidad de examenes totales
examenes_laboratorio_proyectados = (
    examanes_laboratorio_aa + examenes_laboratorio_ac + examenes_laboratorio_urgencia
)

In [39]:
# Muestra la produccion actual
produccion_actual_laboratorio = (
    df_laboratorio_amb.groupby("ano").size() + df_laboratorio_hosp.groupby("ano").size()
)

In [40]:
display(produccion_actual_laboratorio)
display(round(examenes_laboratorio_proyectados))

ano
2015    468546
2016    488454
2017    507761
2018    485015
2019    554398
2020    425257
2021    504242
2022    549272
2023    522606
2024    514917
dtype: int64

2017    1615383.344442
2018    1656238.863318
2019    1698664.683545
2020    1740298.219027
2021     1764352.92768
2022    1779607.337007
2023     1793843.98791
2024    1808702.700736
2025    1821813.198861
2026    1830549.074679
2027    1834445.578433
2028    1834880.880209
2029    1833164.886269
2030    1831124.130853
2031    1829603.301389
2032    1826612.566747
2033    1821417.177781
2034    1813812.772497
2035    1807784.744491
dtype: object

In [41]:
produccion_promedio_laboratorio = produccion_actual_laboratorio[
    produccion_actual_laboratorio.index != 2025
].mean()
crecimiento_laboratorio = examenes_laboratorio_proyectados["2035"] / produccion_promedio_laboratorio
print(f"Aumento de Laboratorio: {crecimiento_laboratorio:.2f} veces")

Aumento de Laboratorio: 3.60 veces


## Farmacia

In [42]:
print("Farmacia - AC - Prescripciones por DCO")
display(prescr_por_dco)

print("Farmacia - AA - Prescripciones por CME")
display(prescr_por_consulta)

Farmacia - AC - Prescripciones por DCO


,count,id_paciente,mean,std,min,25%,50%,75%,max
ano,,,,,,,,,
2020,4008.0,4423.0,3.138212,3.272902,0.0,1.00,2.333333,4.200000,103.500000
2021,4544.0,5103.0,3.354054,3.297278,0.0,1.00,2.500000,4.733333,68.666667
2022,5505.0,6302.0,3.415716,3.301487,0.0,1.25,2.625000,4.700000,73.000000
2023,6183.0,7273.0,3.463834,3.344725,0.0,1.50,2.700000,4.666667,91.000000
2024,6342.0,9779.0,3.861155,3.065502,0.0,1.80,3.000000,5.250000,56.000000


Farmacia - AA - Prescripciones por CME


,count,id_paciente,mean,std,min,25%,50%,75%,max
ano,,,,,,,,,
2020,23673.0,8631.0,0.294377,0.915475,0.0,0.0,0.0,0.083333,48.00
2021,25635.0,14119.0,0.346525,0.860269,0.0,0.0,0.0,0.333333,16.00
2022,30494.0,16510.0,0.362413,1.018257,0.0,0.0,0.0,0.333333,63.00
2023,33188.0,17135.0,0.358337,0.962443,0.0,0.0,0.0,0.333333,55.00
2024,35259.0,17869.0,0.372922,0.943090,0.0,0.0,0.0,0.333333,28.25


In [43]:
# Define constantes
PRESCRIPCIONES_POR_DCO = prescr_por_dco["75%"].max()
PRESCRIPCIONES_POR_CME = 3 # Estandar MINSAL
PRESCRIPCIONES_POR_CU_C1_C2_C3 = 3 # Estandar MINSAL
PRESCRIPCIONES_POR_CU_C4_C5 = 1 # Estandar MINSAL 
metricas_para_proyectar_farmacia = pd.DataFrame(
    {
        "prescripciones_por_dco": [PRESCRIPCIONES_POR_DCO],
        "prescripciones_por_consulta": [PRESCRIPCIONES_POR_CME],
    }
)

In [44]:
# # Obtiene la cantidad de prescripciones
prescripciones_hosp = dco_proyectados * PRESCRIPCIONES_POR_DCO
prescripciones_amb = consultas_medicas_proyectadas * PRESCRIPCIONES_POR_CME
prescripciones_c1_c2_c3 = consultas_c1_c2_c3 * PRESCRIPCIONES_POR_CU_C1_C2_C3
prescripciones_c4_c5 = consultas_c4_c5 * PRESCRIPCIONES_POR_CU_C4_C5

# # Obtiene el total de prescripciones
prescripciones_proyectadas = (
    prescripciones_hosp + prescripciones_amb + prescripciones_c1_c2_c3 + prescripciones_c4_c5
)

In [45]:
produccion_actual_farmacia = (
    df_farmacia_amb.groupby("ano").size() + df_farmacia_hosp.groupby("ano").size()
)
display(produccion_actual_farmacia)

ano
2020    159394
2021    202696
2022    221254
2023    235168
2024    248106
dtype: int64

In [46]:
display(round(prescripciones_proyectadas))

2017    1218105.565569
2018    1249260.391622
2019    1281137.683263
2020    1312216.034696
2021    1330137.652958
2022    1341602.274099
2023    1352292.271753
2024    1363498.540958
2025    1373405.124882
2026    1379955.039562
2027    1382821.654396
2028    1383157.145826
2029    1381881.271243
2030     1380253.68064
2031    1378861.368008
2032    1376515.240499
2033    1372506.884398
2034    1366659.927487
2035    1361920.335768
dtype: object

In [47]:
produccion_promedio_farmacia = produccion_actual_farmacia.mean()
crecimiento_farmacia = prescripciones_proyectadas["2035"] / produccion_promedio_farmacia
print(f"Aumento de farmacia: {crecimiento_farmacia:.2f} veces")

Aumento de farmacia: 6.38 veces


## Imagenologia

In [ ]:
resultados_imag = ua.estimar_unidad_completa(
    df_unidad=df_imag,
    df_unidad_hosp=df_imag_hosp,
    df_unidad_amb=df_imag_amb,
    df_grd=df_grd,
    df_consultas=consultas_medicas,
    casos_hospitalizados_proyectados=casos_hospitalizados_proyectados,
    casos_atencion_abierta_proyectados=casos_atencion_abierta,
    anios_poblacion=ANIOS_POBLACION,
    estimar_examenes_por_paciente_utilizando_glosa=False,
)

## Guardar archivos

In [48]:
# Consolida los casos utilizados para proyectar las unidades de apoyo
resumen_casos_apoyo_dco_cme = pd.DataFrame(
    {
        "DCO Proyectados": dco_proyectados,
        "CME Proyectados": consultas_medicas_proyectadas,
    }
).transpose()
resumen_casos_apoyo_dco_cme["utilizado_en"] = "Laboratorio Clinico - Farmacia"

# Une los casos de apoyo
resumen_casos_apoyo = pd.concat(
    [
        resumen_casos_apoyo_dco_cme,
    ]
).reset_index(names="Trazadoras")

In [49]:
a_guardar = {
    "DCO, CME y Casos Proyecciones": {
        "Casos Utilizados para Proyectar": resumen_casos_apoyo,
    },
    "Laboratorio Clinico": {
        "Exámenes por DCO Históricos": examenes_por_dco_lab.reset_index(),
        "Exámenes por CME Históricos": examenes_por_consulta_lab.reset_index(),
        "Métricas seleccionadas para proyectar": metricas_para_proyectar_laboratorio.reset_index(),
        "Exámenes Proyectados - AC ": examenes_laboratorio_ac.reset_index(),
        "Exámenes Proyectados - AA ": examanes_laboratorio_aa.reset_index(),
        "Exámenes Proyectados - Totales": examenes_laboratorio_proyectados.reset_index(),
    },
    "Farmacia": {
        "Prescripciones por DCO Históricas": prescr_por_dco.reset_index(),
        "Prescripciones por CME Históricas": prescr_por_consulta.reset_index(),
        "Métricas seleccionadas para proyectar": metricas_para_proyectar_farmacia.reset_index(),
        "Prescripciones Proyectadas - AC": prescripciones_hosp.reset_index(),
        "Prescripciones Proyectadas - AA": prescripciones_amb.reset_index(),
        "Prescripciones Proyectada - Total": prescripciones_proyectadas.reset_index(),
    },
}

In [50]:
# Guarda archivos de Apoyo
ruta_a_guardar = "../data/interim/5.0_estimacion_unidades_de_apoyo_RDR.xlsx"
ua.guardar_dict_en_excel(ruta_a_guardar, a_guardar, espacio_filas=3)